In [6]:

from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import pandas as pd
import pickle

# Load your dataset and define the target variable
df = pd.read_csv(r'C:\Users\Tombra\deployment\data.csv')
X = df.drop(columns=['Loan_Status', 'Loan_ID'])
y = df['Loan_Status']

In [10]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import pandas as pd
import pickle

class RawFeats:
    def __init__(self, feats):
        self.feats = feats

    def fit(self, X, y=None):
        pass

    def transform(self, X, y=None):
        return X[self.feats]

    def fit_transform(self, X, y=None):
        self.fit(X)
        return self.transform(X)

# Load your dataset and define the target variable
df = pd.read_csv(r'C:\Users\Tombra\deployment\data.csv')
X = df.drop(columns=['Loan_Status', 'Loan_ID'])
y = df['Loan_Status']

# Define the raw features to be selected
selected_feats = ['Married', 'Education', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term','Gender', 'Self_Employed', 'Property_Area']

# Separate numerical and categorical columns
numerical_columns = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']
categorical_columns = ['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area']

# Convert numerical columns to numeric data type
X[numerical_columns] = X[numerical_columns].apply(pd.to_numeric, errors='coerce')

# Define the preprocessing steps for numerical and categorical data
numerical_preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())  # Use MinMaxScaler to ensure non-negative values
])
categorical_preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(drop='first'))
])

# Create the column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('numerical', numerical_preprocessor, numerical_columns),
        ('categorical', categorical_preprocessor, categorical_columns)
    ])

# Create the feature selector
feature_selector = SelectKBest(score_func=chi2)  # Select features based on chi-square test

# Create the pipeline
pipeline = Pipeline([
    ('raw_feats', RawFeats(feats=selected_feats)),  # Add the RawFeats step with selected feats
    ('preprocessor', preprocessor),
    ('feature_selector', feature_selector),
    ('pca', PCA(n_components=3)),
    ('model', LogisticRegression())
])

# Fit the pipeline on the entire dataset
pipeline.fit(X, y)

# Take a sample row from the dataset
sample_row = X.sample(n=1, random_state=42)

# Predict the probability of being granted a loan for the sample row
probability = pipeline.predict_proba(sample_row)[0][1]

print("Probability of being granted a loan:", probability)


Probability of being granted a loan: 0.7900563392864988


In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import pandas as pd
import pickle

# Load your dataset and define the target variable
df = pd.read_csv(r'C:\Users\Tombra\deployment\data.csv')
X = df.drop(columns=['Loan_Status', 'Loan_ID'])
y = df['Loan_Status']

class RawFeats:
    def __init__(self, feats):
        self.feats = feats

    def fit(self, X, y=None):
        pass

    def transform(self, X, y=None):
        return X[self.feats]

    def fit_transform(self, X, y=None):
        self.fit(X)
        return self.transform(X)



# Define the raw features to be selected
selected_feats = ['Married', 'Education', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term','Gender', 'Self_Employed', 'Property_Area']

# Separate numerical and categorical columns
numerical_columns = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']
categorical_columns = ['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area']

# Convert numerical columns to numeric data type
X[numerical_columns] = X[numerical_columns].apply(pd.to_numeric, errors='coerce')

# Define the preprocessing steps for numerical and categorical data
numerical_preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())  # Use MinMaxScaler to ensure non-negative values
])
categorical_preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(drop='first'))
])

# Create the column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('numerical', numerical_preprocessor, numerical_columns),
        ('categorical', categorical_preprocessor, categorical_columns)
    ])

# Create the feature selector
feature_selector = SelectKBest(score_func=chi2)  # Select features based on chi-square test

# Create the pipeline
pipeline = Pipeline([
    ('raw_feats', RawFeats(feats=selected_feats)),  # Add the RawFeats step with selected feats
    ('preprocessor', preprocessor),
    ('feature_selector', feature_selector),
    ('pca', PCA(n_components=3)),
    ('model', LogisticRegression())
])

# Define the parameter grid for grid search
param_grid = {
    'feature_selector__k': [1, 2, 3],
    'pca__n_components': [2, 3, 4]
}

# Perform grid search
grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=3)
grid_search.fit(X, y)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)



Best Parameters: {'feature_selector__k': 2, 'pca__n_components': 2}
Best Score: 0.6872947553004941


c:\Users\Tombra\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
18 fits failed out of a total of 27.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Tombra\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Tombra\anaconda3\lib\site-packages\sklearn\pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "c:\Users\Tombra\anaconda3\lib\site-packages\sklearn\pipeline.py", line 359, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "c:\Users\Tombra\ana